In [1]:
import pickle, time
import numpy as np
import paddle
import pandas as pd
import paddle.nn as nn
from tqdm import tqdm

t_file = open('data/train_data.pkl', 'rb')
train_data = pickle.load(t_file)
v_file = open('data/eval_data.pkl', 'rb')
eval_data = pickle.load(v_file)

users_df = pd.read_csv('data/users.csv')
items_df = pd.read_csv('data/items.csv')

In [16]:
emb_scale = 128
batch_size = 2048
len_users = len(users_df)
len_items = len(items_df)

In [17]:
class Dataset(paddle.io.Dataset):
    def __init__(self, data):
        self.feature = data[:,0:2]
        self.label = data[:,2]
        pass
    def __getitem__(self, idx):
        return self.feature[idx], self.label[idx]
    def __len__(self):
        return len(self.feature)
    pass

train_dataset = paddle.io.DataLoader(Dataset(train_data),
                               batch_size=batch_size,
                               drop_last=True,
                               shuffle=True)
eval_dataset = paddle.io.DataLoader(Dataset(eval_data),
                               batch_size=batch_size,
                               drop_last=True,
                               shuffle=True)                               

In [37]:
class DSSM(nn.Layer):
    def __init__(self):
        super(DSSM,self).__init__()
        self.users_emb = nn.Embedding(len_users+1, emb_scale)
        self.items_emb = nn.Embedding(len_items+1, emb_scale)
        self.users_fc1 = nn.Linear(emb_scale, 512)
        self.users_fc2 = nn.Linear(512, 300)
        self.users_fc3 = nn.Linear(300, 300)
        self.users_fc4 = nn.Linear(300, 128)
        self.items_fc1 = nn.Linear(emb_scale, 512)
        self.items_fc2 = nn.Linear(512, 300)
        self.items_fc3 = nn.Linear(300, 300)
        self.items_fc4 = nn.Linear(300, 128)
        pass
    def forward(self, input):
        # user tower
        user = self.users_emb(input[:,0])
        user = self.users_fc1(user)
        user = nn.functional.relu(user)
        user = self.users_fc2(user)
        user = nn.functional.relu(user)
        user = self.users_fc3(user)
        user = nn.functional.relu(user)
        user = self.users_fc4(user)
        # item tower
        item = self.items_emb(input[:,1])
        item = self.items_fc1(item)
        item = nn.functional.relu(item)
        item = self.items_fc2(item)
        item = nn.functional.relu(item)
        item = self.items_fc3(item)
        item = nn.functional.relu(item)
        item = self.items_fc4(item)
        # similarity
        x = nn.functional.cosine_similarity(user, item, axis=1)
        x = nn.functional.sigmoid(x)
        return x
    pass
dssm = DSSM()
optim = paddle.optimizer.Adam(parameters=dssm.parameters(), learning_rate=0.01)

In [38]:
train_loss_, train_acc_, eval_loss_, eval_acc_ = [], [], [], []

for epoch in range(10):

    train_loss, train_acc, eval_loss, eval_acc = 0, 0, 0, 0
    dssm.train()

    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        loss.backward()

        optim.step()
        optim.clear_grad()
        train_loss += loss.numpy()[0]
        train_acc += acc.numpy()[0]
    
    train_loss /= (batch_id + 1)
    train_acc /= (batch_id + 1)
    train_loss_.append(train_loss)
    train_acc_.append(train_acc)
    path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), train_loss, train_acc)
    dssm.eval()

    for batch_id, data in enumerate(eval_dataset):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        eval_loss += loss.numpy()[0]
        eval_acc += acc.numpy()[0]

    eval_loss /= (batch_id + 1)
    eval_acc /= (batch_id + 1)
    eval_loss_.append(eval_loss)
    eval_acc_.append(eval_acc)
    print('epoch:{}, loss:{}, acc:{}'.format((epoch + 1), eval_loss, eval_acc))

    # obj = {'model': dssm.state_dict(), 'opt': optim.state_dict(), 'epoch': epoch}
    # path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), eval_loss, eval_acc)
    # paddle.save(obj, path)

pd.DataFrame({
    'train_loss':train_loss_,
    'train_acc':train_acc_, 
    'eval_loss':eval_loss_,
    'eval_acc':eval_acc_
}).to_csv('DSSM/normal.csv')

100%|██████████| 122/122 [00:58<00:00,  2.07it/s]


epoch:1, loss:0.15457591911156973, acc:0.4939778645833333


100%|██████████| 122/122 [00:56<00:00,  2.16it/s]


epoch:2, loss:0.14861618975798288, acc:0.492431640625


100%|██████████| 122/122 [00:56<00:00,  2.15it/s]


...

epoch:8, loss:0.14605757097403207, acc:0.492431640625


100%|██████████| 122/122 [01:06<00:00,  1.85it/s]


epoch:9, loss:0.1460950349768003, acc:0.4938151041666667


100%|██████████| 122/122 [01:04<00:00,  1.88it/s]


epoch:10, loss:0.14642977466185889, acc:0.4935709635416667


In [39]:
class DSSM(nn.Layer):
    def __init__(self):
        super(DSSM,self).__init__()
        self.users_emb = nn.Embedding(len_users+1, emb_scale)
        self.items_emb = nn.Embedding(len_items+1, emb_scale)
        self.users_fc1 = nn.Linear(emb_scale, 512)
        self.users_fc2 = nn.Linear(512, 300)
        self.users_fc3 = nn.Linear(300, 300)
        self.users_fc4 = nn.Linear(300, 128)
        self.items_fc1 = nn.Linear(emb_scale, 512)
        self.items_fc2 = nn.Linear(512, 300)
        self.items_fc3 = nn.Linear(300, 300)
        self.items_fc4 = nn.Linear(300, 128)
        pass
    def forward(self, input):
        # user tower
        user = self.users_emb(input[:,0])
        user = self.users_fc1(user)
        user = nn.functional.relu(user)
        user = self.users_fc2(user)
        user = nn.functional.relu(user)
        user = self.users_fc3(user)
        user = nn.functional.relu(user)
        user = self.users_fc4(user)
        # item tower
        item = self.items_emb(input[:,1])
        item = self.items_fc1(item)
        item = nn.functional.relu(item)
        item = self.items_fc2(item)
        item = nn.functional.relu(item)
        item = self.items_fc3(item)
        item = nn.functional.relu(item)
        item = self.items_fc4(item)
        # similarity
        x = nn.functional.cosine_similarity(user, item, axis=1)
        # x = nn.functional.sigmoid(x)
        return x
    pass
dssm = DSSM()
optim = paddle.optimizer.Adam(parameters=dssm.parameters(), learning_rate=0.01)

train_loss_, train_acc_, eval_loss_, eval_acc_ = [], [], [], []

for epoch in range(10):

    train_loss, train_acc, eval_loss, eval_acc = 0, 0, 0, 0
    dssm.train()

    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        loss.backward()

        optim.step()
        optim.clear_grad()
        train_loss += loss.numpy()[0]
        train_acc += acc.numpy()[0]
    
    train_loss /= (batch_id + 1)
    train_acc /= (batch_id + 1)
    train_loss_.append(train_loss)
    train_acc_.append(train_acc)
    path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), train_loss, train_acc)
    dssm.eval()

    for batch_id, data in enumerate(eval_dataset):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        eval_loss += loss.numpy()[0]
        eval_acc += acc.numpy()[0]

    eval_loss /= (batch_id + 1)
    eval_acc /= (batch_id + 1)
    eval_loss_.append(eval_loss)
    eval_acc_.append(eval_acc)
    print('epoch:{}, loss:{}, acc:{}'.format((epoch + 1), eval_loss, eval_acc))

    # obj = {'model': dssm.state_dict(), 'opt': optim.state_dict(), 'epoch': epoch}
    # path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), eval_loss, eval_acc)
    # paddle.save(obj, path)

pd.DataFrame({
    'train_loss':train_loss_,
    'train_acc':train_acc_, 
    'eval_loss':eval_loss_,
    'eval_acc':eval_acc_
}).to_csv('DSSM/without_sigmoid.csv')

100%|██████████| 122/122 [00:52<00:00,  2.33it/s]


epoch:1, loss:0.1177648976445198, acc:0.4918619791666667


100%|██████████| 122/122 [00:53<00:00,  2.30it/s]


epoch:2, loss:0.109431608269612, acc:0.49267578125


100%|██████████| 122/122 [00:52<00:00,  2.32it/s]


...

epoch:8, loss:0.11155947297811508, acc:0.4922688802083333


100%|██████████| 122/122 [01:10<00:00,  1.73it/s]


epoch:9, loss:0.11418220276633899, acc:0.4915364583333333


100%|██████████| 122/122 [01:14<00:00,  1.64it/s]


epoch:10, loss:0.11421835670868556, acc:0.491455078125


In [40]:
class DSSM(nn.Layer):
    def __init__(self):
        super(DSSM,self).__init__()
        self.users_emb = nn.Embedding(len_users+1, emb_scale)
        self.items_emb = nn.Embedding(len_items+1, emb_scale)
        self.users_fc1 = nn.Linear(emb_scale, 512)
        self.users_fc2 = nn.Linear(512, 300)
        self.users_fc3 = nn.Linear(300, 300)
        self.users_fc4 = nn.Linear(300, 128)
        self.items_fc1 = nn.Linear(emb_scale, 512)
        self.items_fc2 = nn.Linear(512, 300)
        self.items_fc3 = nn.Linear(300, 300)
        self.items_fc4 = nn.Linear(300, 128)
        pass
    def forward(self, input):
        # user tower
        user = self.users_emb(input[:,0])
        user = self.users_fc1(user)
        user = nn.functional.relu(user)
        user = self.users_fc2(user)
        user = nn.functional.relu(user)
        user = self.users_fc3(user)
        user = nn.functional.relu(user)
        user = self.users_fc4(user)
        # item tower
        item = self.items_emb(input[:,1])
        item = self.items_fc1(item)
        item = nn.functional.relu(item)
        item = self.items_fc2(item)
        item = nn.functional.relu(item)
        item = self.items_fc3(item)
        item = nn.functional.relu(item)
        item = self.items_fc4(item)
        # similarity
        x = nn.functional.cosine_similarity(user, item, axis=1)
        x = nn.functional.sigmoid(x)
        return x
    pass
dssm = DSSM()
optim = paddle.optimizer.Adam(parameters=dssm.parameters(), learning_rate=0.001)

train_loss_, train_acc_, eval_loss_, eval_acc_ = [], [], [], []

for epoch in range(10):

    train_loss, train_acc, eval_loss, eval_acc = 0, 0, 0, 0
    dssm.train()

    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        loss.backward()

        optim.step()
        optim.clear_grad()
        train_loss += loss.numpy()[0]
        train_acc += acc.numpy()[0]
    
    train_loss /= (batch_id + 1)
    train_acc /= (batch_id + 1)
    train_loss_.append(train_loss)
    train_acc_.append(train_acc)
    path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), train_loss, train_acc)
    dssm.eval()

    for batch_id, data in enumerate(eval_dataset):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        eval_loss += loss.numpy()[0]
        eval_acc += acc.numpy()[0]

    eval_loss /= (batch_id + 1)
    eval_acc /= (batch_id + 1)
    eval_loss_.append(eval_loss)
    eval_acc_.append(eval_acc)
    print('epoch:{}, loss:{}, acc:{}'.format((epoch + 1), eval_loss, eval_acc))

    # obj = {'model': dssm.state_dict(), 'opt': optim.state_dict(), 'epoch': epoch}
    # path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), eval_loss, eval_acc)
    # paddle.save(obj, path)

pd.DataFrame({
    'train_loss':train_loss_,
    'train_acc':train_acc_, 
    'eval_loss':eval_loss_,
    'eval_acc':eval_acc_
}).to_csv('DSSM/lower_lr.csv')

100%|██████████| 122/122 [00:59<00:00,  2.05it/s]


epoch:1, loss:0.14743067572514215, acc:0.493408203125


100%|██████████| 122/122 [00:58<00:00,  2.10it/s]


epoch:2, loss:0.146359254916509, acc:0.4933268229166667


100%|██████████| 122/122 [01:05<00:00,  1.85it/s]


...

epoch:8, loss:0.14603708187739053, acc:0.4951171875


100%|██████████| 122/122 [01:20<00:00,  1.52it/s]


epoch:9, loss:0.1457853987812996, acc:0.4925944010416667


100%|██████████| 122/122 [01:12<00:00,  1.69it/s]


epoch:10, loss:0.14626964430014291, acc:0.4940592447916667


In [41]:
class DSSM(nn.Layer):
    def __init__(self):
        super(DSSM,self).__init__()
        self.users_emb = nn.Embedding(len_users+1, emb_scale)
        self.items_emb = nn.Embedding(len_items+1, emb_scale)
        self.users_fc1 = nn.Linear(emb_scale, 512)
        self.users_fc2 = nn.Linear(512, 300)
        self.users_fc3 = nn.Linear(300, 300)
        self.users_fc4 = nn.Linear(300, 128)
        self.items_fc1 = nn.Linear(emb_scale, 512)
        self.items_fc2 = nn.Linear(512, 300)
        self.items_fc3 = nn.Linear(300, 300)
        self.items_fc4 = nn.Linear(300, 128)
        pass
    def forward(self, input):
        # user tower
        user = self.users_emb(input[:,0])
        user = self.users_fc1(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc2(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc3(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc4(user)
        # item tower
        item = self.items_emb(input[:,1])
        item = self.items_fc1(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc2(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc3(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc4(item)
        # similarity
        x = nn.functional.cosine_similarity(user, item, axis=1)
        x = nn.functional.sigmoid(x)
        return x
    pass
dssm = DSSM()
optim = paddle.optimizer.Adam(parameters=dssm.parameters(), learning_rate=0.01)

train_loss_, train_acc_, eval_loss_, eval_acc_ = [], [], [], []

for epoch in range(10):

    train_loss, train_acc, eval_loss, eval_acc = 0, 0, 0, 0
    dssm.train()

    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        loss.backward()

        optim.step()
        optim.clear_grad()
        train_loss += loss.numpy()[0]
        train_acc += acc.numpy()[0]
    
    train_loss /= (batch_id + 1)
    train_acc /= (batch_id + 1)
    train_loss_.append(train_loss)
    train_acc_.append(train_acc)
    path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), train_loss, train_acc)
    dssm.eval()

    for batch_id, data in enumerate(eval_dataset):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        eval_loss += loss.numpy()[0]
        eval_acc += acc.numpy()[0]

    eval_loss /= (batch_id + 1)
    eval_acc /= (batch_id + 1)
    eval_loss_.append(eval_loss)
    eval_acc_.append(eval_acc)
    print('epoch:{}, loss:{}, acc:{}'.format((epoch + 1), eval_loss, eval_acc))

    # obj = {'model': dssm.state_dict(), 'opt': optim.state_dict(), 'epoch': epoch}
    # path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), eval_loss, eval_acc)
    # paddle.save(obj, path)

pd.DataFrame({
    'train_loss':train_loss_,
    'train_acc':train_acc_, 
    'eval_loss':eval_loss_,
    'eval_acc':eval_acc_
}).to_csv('DSSM/simplt_model.csv')

100%|██████████| 122/122 [00:33<00:00,  3.59it/s]


epoch:1, loss:0.14963758240143457, acc:0.4949544270833333


100%|██████████| 122/122 [00:34<00:00,  3.56it/s]


epoch:2, loss:0.14667517940203348, acc:0.496826171875


100%|██████████| 122/122 [00:29<00:00,  4.08it/s]


...

epoch:8, loss:0.14668302734692892, acc:0.4943033854166667


100%|██████████| 122/122 [00:29<00:00,  4.20it/s]


epoch:9, loss:0.14700371523698172, acc:0.4942220052083333


100%|██████████| 122/122 [00:31<00:00,  3.93it/s]


epoch:10, loss:0.14699816207091013, acc:0.4927571614583333


In [42]:
class DSSM(nn.Layer):
    def __init__(self):
        super(DSSM,self).__init__()
        self.users_emb = nn.Embedding(len_users+1, emb_scale)
        self.items_emb = nn.Embedding(len_items+1, emb_scale)
        self.users_fc1 = nn.Linear(emb_scale, 512)
        self.users_fc2 = nn.Linear(512, 300)
        self.users_fc3 = nn.Linear(300, 300)
        self.users_fc4 = nn.Linear(300, 128)
        self.items_fc1 = nn.Linear(emb_scale, 512)
        self.items_fc2 = nn.Linear(512, 300)
        self.items_fc3 = nn.Linear(300, 300)
        self.items_fc4 = nn.Linear(300, 128)
        pass
    def forward(self, input):
        # user tower
        user = self.users_emb(input[:,0])
        user = self.users_fc1(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc2(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc3(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc4(user)
        # item tower
        item = self.items_emb(input[:,1])
        item = self.items_fc1(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc2(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc3(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc4(item)
        # similarity
        x = nn.functional.cosine_similarity(user, item, axis=1)
        # x = nn.functional.sigmoid(x)
        return x
    pass
dssm = DSSM()
optim = paddle.optimizer.Adam(parameters=dssm.parameters(), learning_rate=0.01)

train_loss_, train_acc_, eval_loss_, eval_acc_ = [], [], [], []

for epoch in range(10):

    train_loss, train_acc, eval_loss, eval_acc = 0, 0, 0, 0
    dssm.train()

    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        loss.backward()

        optim.step()
        optim.clear_grad()
        train_loss += loss.numpy()[0]
        train_acc += acc.numpy()[0]
    
    train_loss /= (batch_id + 1)
    train_acc /= (batch_id + 1)
    train_loss_.append(train_loss)
    train_acc_.append(train_acc)
    path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), train_loss, train_acc)
    dssm.eval()

    for batch_id, data in enumerate(eval_dataset):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        eval_loss += loss.numpy()[0]
        eval_acc += acc.numpy()[0]

    eval_loss /= (batch_id + 1)
    eval_acc /= (batch_id + 1)
    eval_loss_.append(eval_loss)
    eval_acc_.append(eval_acc)
    print('epoch:{}, loss:{}, acc:{}'.format((epoch + 1), eval_loss, eval_acc))

    # obj = {'model': dssm.state_dict(), 'opt': optim.state_dict(), 'epoch': epoch}
    # path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), eval_loss, eval_acc)
    # paddle.save(obj, path)

pd.DataFrame({
    'train_loss':train_loss_,
    'train_acc':train_acc_, 
    'eval_loss':eval_loss_,
    'eval_acc':eval_acc_
}).to_csv('DSSM/simply_model_without_sigmoid.csv')

100%|██████████| 122/122 [00:28<00:00,  4.28it/s]


epoch:1, loss:0.11470021804173787, acc:0.493408203125


100%|██████████| 122/122 [00:23<00:00,  5.14it/s]


epoch:2, loss:0.11422337964177132, acc:0.4959309895833333


100%|██████████| 122/122 [00:23<00:00,  5.29it/s]


...

epoch:8, loss:0.11684250583251317, acc:0.4931640625


100%|██████████| 122/122 [00:23<00:00,  5.26it/s]


epoch:9, loss:0.11529900878667831, acc:0.4925130208333333


100%|██████████| 122/122 [00:22<00:00,  5.32it/s]


epoch:10, loss:0.11619672924280167, acc:0.4930013020833333


In [43]:
class DSSM(nn.Layer):
    def __init__(self):
        super(DSSM,self).__init__()
        self.users_emb = nn.Embedding(len_users+1, emb_scale)
        self.items_emb = nn.Embedding(len_items+1, emb_scale)
        self.users_fc1 = nn.Linear(emb_scale, 512)
        self.users_fc2 = nn.Linear(512, 300)
        self.users_fc3 = nn.Linear(300, 300)
        self.users_fc4 = nn.Linear(300, 128)
        self.items_fc1 = nn.Linear(emb_scale, 512)
        self.items_fc2 = nn.Linear(512, 300)
        self.items_fc3 = nn.Linear(300, 300)
        self.items_fc4 = nn.Linear(300, 128)
        pass
    def forward(self, input):
        # user tower
        user = self.users_emb(input[:,0])
        user = self.users_fc1(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc2(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc3(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc4(user)
        # item tower
        item = self.items_emb(input[:,1])
        item = self.items_fc1(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc2(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc3(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc4(item)
        # similarity
        x = nn.functional.cosine_similarity(user, item, axis=1)
        x = nn.functional.sigmoid(x)
        return x
    pass
dssm = DSSM()
optim = paddle.optimizer.Adam(parameters=dssm.parameters(), learning_rate=0.001)

train_loss_, train_acc_, eval_loss_, eval_acc_ = [], [], [], []

for epoch in range(10):

    train_loss, train_acc, eval_loss, eval_acc = 0, 0, 0, 0
    dssm.train()

    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        loss.backward()

        optim.step()
        optim.clear_grad()
        train_loss += loss.numpy()[0]
        train_acc += acc.numpy()[0]
    
    train_loss /= (batch_id + 1)
    train_acc /= (batch_id + 1)
    train_loss_.append(train_loss)
    train_acc_.append(train_acc)
    path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), train_loss, train_acc)
    dssm.eval()

    for batch_id, data in enumerate(eval_dataset):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        eval_loss += loss.numpy()[0]
        eval_acc += acc.numpy()[0]

    eval_loss /= (batch_id + 1)
    eval_acc /= (batch_id + 1)
    eval_loss_.append(eval_loss)
    eval_acc_.append(eval_acc)
    print('epoch:{}, loss:{}, acc:{}'.format((epoch + 1), eval_loss, eval_acc))

    # obj = {'model': dssm.state_dict(), 'opt': optim.state_dict(), 'epoch': epoch}
    # path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), eval_loss, eval_acc)
    # paddle.save(obj, path)

pd.DataFrame({
    'train_loss':train_loss_,
    'train_acc':train_acc_, 
    'eval_loss':eval_loss_,
    'eval_acc':eval_acc_
}).to_csv('DSSM/simply_model_lower_lr.csv')

100%|██████████| 122/122 [00:24<00:00,  4.95it/s]


epoch:1, loss:0.14715813348690668, acc:0.4928385416666667


100%|██████████| 122/122 [00:22<00:00,  5.46it/s]


epoch:2, loss:0.1463107317686081, acc:0.4925130208333333


100%|██████████| 122/122 [00:16<00:00,  7.55it/s]


...

epoch:8, loss:0.14622468004624048, acc:0.4917805989583333


100%|██████████| 122/122 [00:22<00:00,  5.39it/s]


epoch:9, loss:0.14637544751167297, acc:0.4947916666666667


100%|██████████| 122/122 [00:22<00:00,  5.49it/s]


epoch:10, loss:0.14580203841129938, acc:0.4918619791666667


In [44]:
class DSSM(nn.Layer):
    def __init__(self):
        super(DSSM,self).__init__()
        self.users_emb = nn.Embedding(len_users+1, emb_scale)
        self.items_emb = nn.Embedding(len_items+1, emb_scale)
        self.users_fc1 = nn.Linear(emb_scale, 512)
        self.users_fc2 = nn.Linear(512, 300)
        self.users_fc3 = nn.Linear(300, 300)
        self.users_fc4 = nn.Linear(300, 128)
        self.items_fc1 = nn.Linear(emb_scale, 512)
        self.items_fc2 = nn.Linear(512, 300)
        self.items_fc3 = nn.Linear(300, 300)
        self.items_fc4 = nn.Linear(300, 128)
        pass
    def forward(self, input):
        # user tower
        user = self.users_emb(input[:,0])
        user = self.users_fc1(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc2(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc3(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc4(user)
        # item tower
        item = self.items_emb(input[:,1])
        item = self.items_fc1(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc2(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc3(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc4(item)
        # similarity
        x = nn.functional.cosine_similarity(user, item, axis=1)
        # x = nn.functional.sigmoid(x)
        return x
    pass
dssm = DSSM()
optim = paddle.optimizer.Adam(parameters=dssm.parameters(), learning_rate=0.001)

train_loss_, train_acc_, eval_loss_, eval_acc_ = [], [], [], []

for epoch in range(10):

    train_loss, train_acc, eval_loss, eval_acc = 0, 0, 0, 0
    dssm.train()

    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        loss.backward()

        optim.step()
        optim.clear_grad()
        train_loss += loss.numpy()[0]
        train_acc += acc.numpy()[0]
    
    train_loss /= (batch_id + 1)
    train_acc /= (batch_id + 1)
    train_loss_.append(train_loss)
    train_acc_.append(train_acc)
    path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), train_loss, train_acc)
    dssm.eval()

    for batch_id, data in enumerate(eval_dataset):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        eval_loss += loss.numpy()[0]
        eval_acc += acc.numpy()[0]

    eval_loss /= (batch_id + 1)
    eval_acc /= (batch_id + 1)
    eval_loss_.append(eval_loss)
    eval_acc_.append(eval_acc)
    print('epoch:{}, loss:{}, acc:{}'.format((epoch + 1), eval_loss, eval_acc))

    # obj = {'model': dssm.state_dict(), 'opt': optim.state_dict(), 'epoch': epoch}
    # path = 'DSSM/{}_epoch{}_loss{}_acc{}_model.pdparams'.format(int(time.time()), (epoch + 1), eval_loss, eval_acc)
    # paddle.save(obj, path)

pd.DataFrame({
    'train_loss':train_loss_,
    'train_acc':train_acc_, 
    'eval_loss':eval_loss_,
    'eval_acc':eval_acc_
}).to_csv('DSSM/simply_model_without_sigmoid_lower_lr.csv')

100%|██████████| 122/122 [00:23<00:00,  5.11it/s]


epoch:1, loss:0.1102434868613879, acc:0.4969889322916667


100%|██████████| 122/122 [00:23<00:00,  5.25it/s]


epoch:2, loss:0.10920329764485359, acc:0.4937337239583333


100%|██████████| 122/122 [00:23<00:00,  5.13it/s]


...

epoch:8, loss:0.11506962403655052, acc:0.4931640625


100%|██████████| 122/122 [00:23<00:00,  5.19it/s]


epoch:9, loss:0.11377445111672084, acc:0.49365234375


100%|██████████| 122/122 [00:24<00:00,  4.92it/s]


epoch:10, loss:0.11446983988086383, acc:0.4921875
